In [1]:
import os

import numpy as np
import pandas as pd
import joblib

import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report

some useful functions **TODO**: turn to script and import for cleanliness

In [2]:
def getFiles(organName, listDir, prefix=None):
    """
    Generate list of files associa-
    ted with each organ of interest
    in the list of files.
    --------------------------
    Input:
        organ_list: list of organs
                    str
        listDir:    list of files  
                    to search
        prefix:     OS path prefix
    Output:
        lists:      list of files 
                    associated with
                    organ
    """
    # get the files from listDir that contain 'organName'
    matching = [file for file in listDir if organName in file]
    
    # append the path prefix so that files are accessible
    if prefix:
        matchingFixed = [prefix + file for file in matching]
        return matchingFixed
    
    else:
        return matching

In [3]:
def filestoDF(file_list):
    """
    Return a list of data frames from the given
    list of files.
    """
    dfs = [pd.read_csv(file, low_memory=False) for file in file_list]
    return dfs

Classifier functions and classes

In [4]:
class RFClassifier:
    def __init__(self, labels, multilabel=False):
        # boolean, if multilabel (T/F)
        self.multilabel = multilabel 
        # initialize model (RF), method below
        self.model = self.initialize_model() 
        
        # set labels ['Present', 'Absent'] or multilabels
        self.labels = labels 
        # idx each label in a dictionary key: label, val: idx
        self.label2idx = {l: i for i, l in enumerate(self.labels)}

    def initialize_model(self):
        # model is random forest, n=-1 means use all available
        # processors (parallel)
        model = RandomForestClassifier(verbose=False, n_jobs=-1)
        # if true in multilabel, 
        if self.multilabel:
            # make it multilabel: one classifier (assign a label
            # to data point) per target, estimator (first param)
            # is the model at hand (random forest here)
            model = MultiOutputClassifier(model, n_jobs=-1)
        
        model = make_pipeline(TfidfVectorizer(ngram_range=(1, 5)), model)
        return model

    def fit(self, X, y):
        # first fix depending on multilabel or binary
        
        # if not False: then its true, then binary
        if not self.multilabel:
            y = np.ravel(y)
            # idx at label
            y = [self.label2idx[i] for i in y]
        # otherwise multilabel
        else:
            multilabel_idx = np.zeros(shape=(len(X), len(self.labels)))
            for i, cur_labels in enumerate(y):
                for cur_label in cur_labels:
                    multilabel_idx[i, self.label2idx.get(cur_label)] = 1
            y = multilabel_idx
            
        # now call on random forest fit method once y is appropriately
        # constructed
        return self.model.fit(X, y)

    def predict(self, X):
        # get model probability predictions
        probabilities = self.model.predict_proba(X)
        
        # if binary
        if not self.multilabel:
            # idx of the max probability in label
            predicted_label_indices = np.argmax(probabilities, axis=1)
            # predicted scores for each
            predicted_scores = probabilities[np.arange(len(predicted_label_indices)), predicted_label_indices]
        # else it is multilabel
        else:
            predicted_label_indices = self.model.predict(X)
            # redefine the var probabilities, this is mainly to deal with the prob outputs of multilabel
            # should work bc the model was already altered to be multilabel
            probabilities = [np.max(i, axis=1) for i in probabilities]
            probabilities = np.stack(probabilities, axis=1)

            # Using maximum confidence over all labels as the classifier confidence, since random forest
            predicted_scores = np.max(probabilities, axis=1)
            
        predictions, scores = list(), list()
        for idx, score in zip(predicted_label_indices, predicted_scores):
            if not self.multilabel:
                predicted_labels = [self.labels[idx]]
            else:
                predicted_labels = idx # [self.labels[i] for i, j in enumerate(idx) if int(j) == 1]
            predictions.append(predicted_labels)

        return predictions, predicted_scores

In [5]:
def predictFiles(model, fileList, dfList, classes=['Present', 'Absent'], impression=False):
    """
    Predict from fileList and dfList.
    List of files provided to keep track of
    file names. DF list provided since
    it was previously computed.
    
    Output is a list of DFs.
    """
    
    # save your dataframes, perhaps in csv later
    finalDFS = []
    for i, frame in enumerate(dfList):
        
        # file name keeping
        split0 = fileList[i].split('/')[-1].split('.')[0]
        split1 = split0.split('_')[0:2]
        name = ' '.join(split1)

        # X data from df: text is either impression + org note, org note
        if impression:
            X_dev = frame['Impression and Note']
            ids = frame['idx'].tolist()
        elif not impression:
            X_dev = frame['organ_sentence']
            ids = frame['idx'].tolist()
        
        # gold labels and str from list
        if len(classes) == 2:
            y_dev = frame['organ_label']
            y_dev = [i[2:-2] for i in y_dev]
        elif len(classes) > 2:
            y_dev = frame['disease_label']
            y_dev = [i[2:-2] for i in y_dev]

        # get preds and str from list
        preds, pred_probs = model.predict(X_dev)
        preds = [i[0] for i in preds]
        
        # results --> df 
        res = pd.DataFrame({'idx': ids,
                            "text_impression_note": X_dev,
                            "Predictions": preds,
                            "Actual": y_dev,
                            "Prediction Probabilities": pred_probs})
        finalDFS.append(res)

        # metrics
        print("****** Classification report for: ", name.upper()+ " *********")
        clf_rep = classification_report(y_dev, preds, labels = classes, zero_division=0)
        print(clf_rep)
        print("      ")
        
    return finalDFS

### Train: Binary
Data import

In [6]:
dirPath = '../data_200/data_silvia/'
files = os.listdir(dirPath)

# get train and dev files
trainFiles = getFiles('train', files, prefix = dirPath)
devFiles = getFiles('dev', files, prefix = dirPath)

# liver and pancreas
trainFiles_Liver = sorted(getFiles('Liver', trainFiles))
devFiles_Liver = sorted(getFiles('Liver', devFiles))
trainFiles_Pancreas = sorted(getFiles('Pancreas', trainFiles))
devFiles_Pancreas = sorted(getFiles('Pancreas', devFiles))

# files to dfs (liver and pancreas separate)
trainDF_Liver = filestoDF(trainFiles_Liver)
devDF_Liver = filestoDF(devFiles_Liver)
trainDF_Pancreas = filestoDF(trainFiles_Pancreas)
devDF_Pancreas = filestoDF(devFiles_Pancreas)

**Liver Data Only**

In [7]:
# file order: abnormal finds, indeterminate nods, prev surg, dis loc
X_trainIMP = trainDF_Liver[0]['Impression and Note'].tolist() + trainDF_Liver[1]['Impression and Note'].tolist() + trainDF_Liver[2]['Impression and Note'].tolist() + trainDF_Liver[3]['Impression and Note'].tolist()

yIMP_0 = [i[2:len(i)-2] for i in trainDF_Liver[0]['disease_label'].tolist()]
yIMP_1 = [i[2:len(i)-2] for i in trainDF_Liver[1]['disease_label'].tolist()]
yIMP_2 = [i[2:len(i)-2] for i in trainDF_Liver[2]['disease_label'].tolist()]
yIMP_3 = [i[2:len(i)-2] for i in trainDF_Liver[3]['disease_label'].tolist()]
y_trainIMP = yIMP_0 + yIMP_1 + yIMP_2 + yIMP_3

X_trainNOT = trainDF_Liver[0]['organ_sentence'].tolist() + trainDF_Liver[1]['organ_sentence'].tolist() + trainDF_Liver[2]['organ_sentence'].tolist() + trainDF_Liver[3]['organ_sentence'].tolist()

yNOT_0 = [i[2:len(i)-2] for i in trainDF_Liver[0]['organ_label'].tolist()]
yNOT_1 = [i[2:len(i)-2] for i in trainDF_Liver[1]['organ_label'].tolist()]
yNOT_2 = [i[2:len(i)-2] for i in trainDF_Liver[2]['organ_label'].tolist()]
yNOT_3 = [i[2:len(i)-2] for i in trainDF_Liver[3]['organ_label'].tolist()]
# all notes combined
y_trainNOT = yNOT_0 + yNOT_1 + yNOT_2 + yNOT_3


# may need to re-structure this below or here to properly test eveything. 
X_devIMP = devDF_Liver[0]['Impression and Note']
X_devNOT = devDF_Liver[0]['organ_sentence']

y_devIMP = devDF_Liver[0]['disease_label']
y_devNOT = devDF_Liver[0]['organ_label']

**Train on organ notes** (`Present`/`Absent`).

In [8]:
LiverClassifier_BNOT = RFClassifier(['Present', 'Absent'], multilabel = False)
# now we combine all organ_sentence notes for all Liver issues (abnormal findings, indeterminate nods..etc)
# with corresponding binary labels -- see cell above
LiverClassifier_BNOT.fit(X_trainNOT, y_trainNOT)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [9]:
# save model
joblib.dump(LiverClassifier_BNOT, "models/RandomForest_LiverNOTE_binary.joblib")

['models/RandomForest_LiverNOTE_binary.joblib']

**Train on organ notes + impression** (`Present`/`Absent`): *will the addition of the impression text improve scores?*

In [10]:
LiverClassifier_BIMP = RFClassifier(['Present', 'Absent'], multilabel = False)
# still keeping the same binary labels of present/absent
LiverClassifier_BIMP.fit(X_trainIMP, y_trainNOT)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [11]:
# save model 
joblib.dump(LiverClassifier_BIMP, "models/RandomForest_LiverIMP_binary.joblib")

['models/RandomForest_LiverIMP_binary.joblib']

**Organ Note predictions**: Predict on all dev files

In [12]:
# now predict on dev files (abnormal finds)
abnormalFinds1 = predictFiles(LiverClassifier_BNOT,[devFiles_Liver[0]], [devDF_Liver[0]])
# predict indeterminate nodes
indeterNods1 = predictFiles(LiverClassifier_BNOT, [devFiles_Liver[1]], [devDF_Liver[1]])
# predict previous surgery
prevSurg1 = predictFiles(LiverClassifier_BNOT, [devFiles_Liver[2]], [devDF_Liver[2]])
# predict disease location
diseaseLoc1 = predictFiles(LiverClassifier_BNOT, [devFiles_Liver[3]], [devDF_Liver[3]])

****** Classification report for:  ABNORMAL FINDINGS LIVER *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00        20
      Absent       0.50      0.95      0.66        21

    accuracy                           0.49        41
   macro avg       0.25      0.48      0.33        41
weighted avg       0.26      0.49      0.34        41

      
****** Classification report for:  INDETERMINATE NODULES LIVER *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00         8
      Absent       0.80      0.97      0.88        33

    accuracy                           0.78        41
   macro avg       0.40      0.48      0.44        41
weighted avg       0.64      0.78      0.71        41

      
****** Classification report for:  PREVIOUS SURGERIES LIVER *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00         2
      Absen

**Impression + Note Predictions**

In [13]:
# predict on dev abnormal finds
abnormalFinds2 = predictFiles(LiverClassifier_BIMP,[devFiles_Liver[0]],[devDF_Liver[0]], impression=True)
# predict indeterminate nodes
indeterNods2 = predictFiles(LiverClassifier_BIMP, [devFiles_Liver[1]], [devDF_Liver[1]], impression=True)
# predict previous surgery
prevSurg2 = predictFiles(LiverClassifier_BIMP, [devFiles_Liver[2]], [devDF_Liver[2]], impression=True)
# predict disease location
diseaseLoc2 = predictFiles(LiverClassifier_BIMP, [devFiles_Liver[3]], [devDF_Liver[3]], impression=True)

****** Classification report for:  ABNORMAL FINDINGS LIVER *********
              precision    recall  f1-score   support

     Present       1.00      0.10      0.18        20
      Absent       0.54      1.00      0.70        21

    accuracy                           0.56        41
   macro avg       0.77      0.55      0.44        41
weighted avg       0.76      0.56      0.45        41

      
****** Classification report for:  INDETERMINATE NODULES LIVER *********
              precision    recall  f1-score   support

     Present       1.00      0.25      0.40         8
      Absent       0.85      1.00      0.92        33

    accuracy                           0.85        41
   macro avg       0.92      0.62      0.66        41
weighted avg       0.88      0.85      0.82        41

      
****** Classification report for:  PREVIOUS SURGERIES LIVER *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00         2
      Absen

Interesting! Not excellent but there was some improvement. Increased training data by training on all organ notes with corresponding labels. Same applied to impression + organ notes.
Now Previous surgeries performs rel. well as oppossed to Abnormal findings (using `LIT` for this).

### Train: Multilabel

**Train with Impression + Organ Notes** Multilabel [`Disease Stability`, `No evidence of disease`, etc]

Note that I am not trainig with only organ notes, since less features (organ notes only) will not necessarily guarantee a good performance from ML perspective. In other words, I'd be expecting that less text (organ specific notes) could successfully predict the multiclass labels.
- Have also expanded training data to include all impression + organ notes for liver.

In [14]:
multiclass = ['Disease Stability','No evidence of disease',
              'Progression', 'Unclear', 'Mixed', 'Treatment response']

# M: multilabel, IMP: impression
LiverClassifier_MIMP = RFClassifier(multiclass, multilabel=False)
LiverClassifier_MIMP.fit(X_trainIMP, y_trainIMP)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [15]:
# save model
joblib.dump(LiverClassifier_MIMP, "models/RandomForest_LiverIMP_multi.joblib")

['models/RandomForest_LiverIMP_multi.joblib']

**Predict on Dev Impression + Note Files**

In [16]:
# predict on dev abnormal finds
abnormalFinds3 = predictFiles(LiverClassifier_MIMP,[devFiles_Liver[0]],
                              [devDF_Liver[0]], classes = multiclass[:5],
                              impression=True)
# predict indeterminate nodes
indeterNods3 = predictFiles(LiverClassifier_MIMP, [devFiles_Liver[1]],
                            [devDF_Liver[1]], classes = multiclass[:5],
                            impression=True)
# predict previous surgery
prevSurg3 = predictFiles(LiverClassifier_MIMP, [devFiles_Liver[2]],
                         [devDF_Liver[2]], classes = multiclass[:5],
                         impression=True)
# predict disease location
diseaseLoc3 = predictFiles(LiverClassifier_MIMP, [devFiles_Liver[3]],
                           [devDF_Liver[3]], classes = multiclass[:5],
                          impression=True)

****** Classification report for:  ABNORMAL FINDINGS LIVER *********
                        precision    recall  f1-score   support

     Disease Stability       0.50      1.00      0.67         3
No evidence of disease       0.76      0.84      0.80        19
           Progression       0.67      0.55      0.60        11
               Unclear       0.60      0.43      0.50         7
                 Mixed       0.00      0.00      0.00         1

              accuracy                           0.68        41
             macro avg       0.51      0.56      0.51        41
          weighted avg       0.67      0.68      0.67        41

      
****** Classification report for:  INDETERMINATE NODULES LIVER *********
                        precision    recall  f1-score   support

     Disease Stability       0.50      1.00      0.67         3
No evidence of disease       0.76      0.84      0.80        19
           Progression       0.67      0.55      0.60        11
               

Weird how f1 averages do not change, the inclusion of the organ notes does not seem to influence performance.
**TODO:** Double check work to see if there are any errors, clean up.

### Pancreas Data

In [17]:
# file order: abnormal finds, indeterminate nods, prev surg, dis loc
X_trainIMP = trainDF_Pancreas[0]['Impression and Note'].tolist() + trainDF_Pancreas[1]['Impression and Note'].tolist() + trainDF_Pancreas[2]['Impression and Note'].tolist() + trainDF_Pancreas[3]['Impression and Note'].tolist()

yIMP_0 = [i[2:len(i)-2] for i in trainDF_Pancreas[0]['disease_label'].tolist()]
yIMP_1 = [i[2:len(i)-2] for i in trainDF_Pancreas[1]['disease_label'].tolist()]
yIMP_2 = [i[2:len(i)-2] for i in trainDF_Pancreas[2]['disease_label'].tolist()]
yIMP_3 = [i[2:len(i)-2] for i in trainDF_Pancreas[3]['disease_label'].tolist()]
y_trainIMP = yIMP_0 + yIMP_1 + yIMP_2 + yIMP_3

X_trainNOT = trainDF_Pancreas[0]['organ_sentence'].tolist() + trainDF_Pancreas[1]['organ_sentence'].tolist() + trainDF_Pancreas[2]['organ_sentence'].tolist() + trainDF_Pancreas[3]['organ_sentence'].tolist()

yNOT_0 = [i[2:len(i)-2] for i in trainDF_Pancreas[0]['organ_label'].tolist()]
yNOT_1 = [i[2:len(i)-2] for i in trainDF_Pancreas[1]['organ_label'].tolist()]
yNOT_2 = [i[2:len(i)-2] for i in trainDF_Pancreas[2]['organ_label'].tolist()]
yNOT_3 = [i[2:len(i)-2] for i in trainDF_Pancreas[3]['organ_label'].tolist()]
y_trainNOT = yNOT_0 + yNOT_1 + yNOT_2 + yNOT_3


# may need to re-structure this below or here to properly test eveything. 
X_devIMP = devDF_Pancreas[0]['Impression and Note']
X_devNOT = devDF_Pancreas[0]['organ_sentence']

y_devIMP = devDF_Pancreas[0]['disease_label']
y_devNOT = devDF_Pancreas[0]['organ_label']

### Train Pancreas Data

**Train on Pancreas Organ Notes**: `Present`/`Absent`

In [18]:
PancreasClassifier_BNOT = RFClassifier(['Present', 'Absent'], multilabel = False)
# now we combine all organ_sentence notes for all Liver issues (abnormal findings, indeterminate nods..etc)
# with corresponding binary labels -- see cell above
PancreasClassifier_BNOT.fit(X_trainNOT, y_trainNOT)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [19]:
# save model
joblib.dump(PancreasClassifier_BNOT, "models/RandomForest_PancreasNOTE_binary.joblib")

['models/RandomForest_PancreasNOTE_binary.joblib']

**Train with Pancreas Organ Notes + Impression**: `Present`/`Absent`

In [20]:
PancreasClassifier_BIMP = RFClassifier(['Present', 'Absent'], multilabel = False)
# still keeping the same binary labels of present/absent
PancreasClassifier_BIMP.fit(X_trainIMP, y_trainNOT)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [21]:
joblib.dump(PancreasClassifier_BIMP, "models/RandomForest_PancreasIMP_binary.joblib")

['models/RandomForest_PancreasIMP_binary.joblib']

**Train with Pancreas Organ + Impression Note**: `Multi-label`

In [22]:
multiclass = ['Disease Stability','No evidence of disease',
              'Progression', 'Unclear', 'Mixed', 'Treatment response']

# M: multilabel, IMP: impression
PancreasClassifier_MIMP = RFClassifier(multiclass, multilabel=False)
PancreasClassifier_MIMP.fit(X_trainIMP, y_trainIMP)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 5))),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=False))])

In [23]:
joblib.dump(PancreasClassifier_MIMP, "models/RandomForest_PancreasIMP_multi.joblib")

['models/RandomForest_PancreasIMP_multi.joblib']

### Predict

**Predict Pancreas Dev Data**: Organ Note only Binary prediction

In [24]:
# now predict on dev files (abnormal finds)
abnormalFinds1_Panc = predictFiles(PancreasClassifier_BNOT,[devFiles_Pancreas[0]], [devDF_Pancreas[0]])
# predict indeterminate nodes
indeterNods1_Panc = predictFiles(PancreasClassifier_BNOT, [devFiles_Pancreas[1]], [devDF_Pancreas[1]])
# predict previous surgery
prevSurg1_Panc = predictFiles(PancreasClassifier_BNOT, [devFiles_Pancreas[2]], [devDF_Pancreas[2]])
# predict disease location
diseaseLoc1_Panc = predictFiles(PancreasClassifier_BNOT, [devFiles_Pancreas[3]], [devDF_Pancreas[3]])

****** Classification report for:  ABNORMAL FINDINGS PANCREAS *********
              precision    recall  f1-score   support

     Present       0.57      0.17      0.27        23
      Absent       0.44      0.83      0.58        18

    accuracy                           0.46        41
   macro avg       0.51      0.50      0.42        41
weighted avg       0.51      0.46      0.40        41

      
****** Classification report for:  INDETERMINATE NODULES PANCREAS *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00         0
      Absent       1.00      0.83      0.91        41

    accuracy                           0.83        41
   macro avg       0.50      0.41      0.45        41
weighted avg       1.00      0.83      0.91        41

      
****** Classification report for:  PREVIOUS SURGERIES PANCREAS *********
              precision    recall  f1-score   support

     Present       0.43      0.19      0.26        16
  

**Predict Pancreas Dev Data**: Organ Note + Impression -- Binary

In [25]:
# now predict on dev files (abnormal finds)
abnormalFinds2_Panc = predictFiles(PancreasClassifier_BIMP,[devFiles_Pancreas[0]], [devDF_Pancreas[0]], impression=True)
# predict indeterminate nodes
indeterNods2_Panc = predictFiles(PancreasClassifier_BIMP, [devFiles_Pancreas[1]], [devDF_Pancreas[1]], impression=True)
# predict previous surgery
prevSurg2_Panc = predictFiles(PancreasClassifier_BIMP, [devFiles_Pancreas[2]], [devDF_Pancreas[2]], impression=True)
# predict disease location
diseaseLoc2_Panc = predictFiles(PancreasClassifier_BIMP, [devFiles_Pancreas[3]], [devDF_Pancreas[3]], impression=True)

****** Classification report for:  ABNORMAL FINDINGS PANCREAS *********
              precision    recall  f1-score   support

     Present       1.00      0.04      0.08        23
      Absent       0.45      1.00      0.62        18

    accuracy                           0.46        41
   macro avg       0.72      0.52      0.35        41
weighted avg       0.76      0.46      0.32        41

      
****** Classification report for:  INDETERMINATE NODULES PANCREAS *********
              precision    recall  f1-score   support

     Present       0.00      0.00      0.00         0
      Absent       1.00      0.98      0.99        41

    accuracy                           0.98        41
   macro avg       0.50      0.49      0.49        41
weighted avg       1.00      0.98      0.99        41

      
****** Classification report for:  PREVIOUS SURGERIES PANCREAS *********
              precision    recall  f1-score   support

     Present       1.00      0.06      0.12        16
  

**Predict Pancreas Dev Data**: Organ Note + Impression -- Multilabel

In [26]:
# now predict on dev files (abnormal finds)
abnormalFinds3_Panc = predictFiles(PancreasClassifier_MIMP,[devFiles_Pancreas[0]], [devDF_Pancreas[0]],
                                   classes=multiclass[:5], impression=True)
# predict indeterminate nodes
indeterNods3_Panc = predictFiles(PancreasClassifier_MIMP, [devFiles_Pancreas[1]], [devDF_Pancreas[1]],
                                classes=multiclass[:5], impression=True)
# predict previous surgery
prevSurg3_Panc = predictFiles(PancreasClassifier_MIMP, [devFiles_Pancreas[2]], [devDF_Pancreas[2]],
                             classes=multiclass[:5], impression=True)
# predict disease location
diseaseLoc3_Panc = predictFiles(PancreasClassifier_MIMP, [devFiles_Pancreas[3]], [devDF_Pancreas[3]],
                               classes=multiclass[:5], impression=True)

****** Classification report for:  ABNORMAL FINDINGS PANCREAS *********
                        precision    recall  f1-score   support

     Disease Stability       0.40      0.67      0.50         3
No evidence of disease       0.86      0.95      0.90        19
           Progression       0.64      0.82      0.72        11
               Unclear       0.00      0.00      0.00         7
                 Mixed       0.00      0.00      0.00         1

              accuracy                           0.71        41
             macro avg       0.38      0.49      0.42        41
          weighted avg       0.60      0.71      0.65        41

      
****** Classification report for:  INDETERMINATE NODULES PANCREAS *********
                        precision    recall  f1-score   support

     Disease Stability       0.40      0.67      0.50         3
No evidence of disease       0.86      0.95      0.90        19
           Progression       0.64      0.82      0.72        11
         